# Building a Smart Enterprise Chatbot Across Departments with Cortex Agents

## Create the database, tables and warehouse

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
CREATE DATABASE IF NOT EXISTS ENTERPRISE_DATA_DB;

CREATE SCHEMA IF NOT EXISTS ENTERPRISE_DATA_SCHEMA;

CREATE WAREHOUSE IF NOT EXISTS ENTERPRISE_CORTEX_WH WITH
     WAREHOUSE_SIZE='X-SMALL'
     AUTO_SUSPEND = 120
     AUTO_RESUME = TRUE
     INITIALLY_SUSPENDED=TRUE;

 USE WAREHOUSE ENTERPRISE_CORTEX_WH;

You will use a sample dataset. Download the files directly from your browser by following this link:

[Sample Dataset](https://github.com/anudeeppendyala-1994/sf_cortex_june2025_workshop/tree/main/Sample_data)


## Load data into Snowflake stage

In [ ]:
CREATE OR REPLACE STAGE ENTERPRISE_DATA_DB.ENTERPRISE_DATA_SCHEMA.LANDING_STG
    DIRECTORY = (ENABLE = TRUE)
    ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE');

Now upload the dataset. You can upload the dataset in Snowsight or using SQL. To upload in Snowsight:

1. Sign in to Snowsight.

2. Select Data in the left-side navigation menu.

3. Select your database ENTERPRISE_DATA_DB.

4. Select your schema ENTERPRISE_DATA_SCHEMA.

5. Select Stages and select landing_stg.

6. On the top right, Select the + Files button.

7. Drag and drop files into the UI or select Browse to choose a file from the dialog window.

8. Select Upload your files with a folder to it. like Sales/sales.pdf, HR/HR.pdf, Marketing/marketing.pdf

In [ ]:
ls @ENTERPRISE_DATA_DB.ENTERPRISE_DATA_SCHEMA.landing_stg

In [ ]:
SELECT CURRENT_DATABASE()||'.'||CURRENT_SCHEMA();
-- if not ENTERPRISE_DATA_DB database then execute below statement
--USE SCHEMA ENTERPRISE_DATA_DB.ENTERPRISE_DATA_SCHEMA;

HR Database & Schema Setup

In [ ]:
-- =============================================
-- HR Database Setup
-- =============================================

CREATE OR REPLACE DATABASE HR_DB;
USE DATABASE HR_DB;

-- Create schemas
CREATE OR REPLACE SCHEMA HR_DB.RAW;
CREATE OR REPLACE SCHEMA HR_DB.ANALYTICS;


CREATE OR REPLACE STAGE HR_DB.RAW.LANDING_STG
    DIRECTORY = (ENABLE = TRUE)
    ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE');



-- =============================================
-- Sales Database Setup
-- =============================================

CREATE OR REPLACE DATABASE SALES_DB;
USE DATABASE SALES_DB;

-- Create schemas
CREATE OR REPLACE SCHEMA SALES_DB.RAW;
CREATE OR REPLACE SCHEMA SALES_DB.ANALYTICS;

CREATE OR REPLACE STAGE SALES_DB.RAW.LANDING_STG
    DIRECTORY = (ENABLE = TRUE)
    ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE');


-- =============================================
-- Marketing Database Setup
-- =============================================

CREATE OR REPLACE DATABASE MARKETING_DB;
USE DATABASE MARKETING_DB;

-- Create schemas
CREATE OR REPLACE SCHEMA MARKETING_DB.RAW;
CREATE OR REPLACE SCHEMA MARKETING_DB.ANALYTICS;

CREATE OR REPLACE STAGE MARKETING_DB.RAW.LANDING_STG
    DIRECTORY = (ENABLE = TRUE)
    ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE');

In [ ]:
USE DATABASE HR_DB;

USE SCHEMA RAW;

CREATE OR REPLACE FILE FORMAT HR_DB.RAW.my_csv_format
  TYPE = csv
  FIELD_OPTIONALLY_ENCLOSED_BY = '"'
  VALIDATE_UTF8 = TRUE
  PARSE_HEADER = TRUE
  ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE;

-- Auto create table with Schema Detection and enable Schema Evolution
CREATE OR REPLACE TABLE HR_DB.RAW.EMPLOYEES
    USING TEMPLATE (
        SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
          FROM TABLE(
            INFER_SCHEMA(        
            LOCATION=>'@HR_DB.RAW.landing_stg/employees.csv'
            , FILE_FORMAT => 'my_csv_format'
            , IGNORE_CASE => TRUE

            
        )
    ))
ENABLE_SCHEMA_EVOLUTION = TRUE
;

-- Auto create table with Schema Detection and enable Schema Evolution
CREATE OR REPLACE TABLE HR_DB.RAW.DEPARTMENT
    USING TEMPLATE (
        SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
        FROM TABLE(
            INFER_SCHEMA(        
            LOCATION=>'@HR_DB.RAW.landing_stg/departments.csv'
            , FILE_FORMAT => 'my_csv_format'
            , IGNORE_CASE => TRUE
        )
    ))
ENABLE_SCHEMA_EVOLUTION = TRUE;

-- Auto create table with Schema Detection and enable Schema Evolution
CREATE OR REPLACE TABLE HR_DB.RAW.PERFORMANCE_REVIEWS
    USING TEMPLATE (
        SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
        FROM TABLE(
            INFER_SCHEMA(        
            LOCATION=>'@HR_DB.RAW.landing_stg/Performance_reviews.csv'
            , FILE_FORMAT => 'my_csv_format'
            , IGNORE_CASE => TRUE
        )
    ))
ENABLE_SCHEMA_EVOLUTION = TRUE;


ALTER TABLE HR_DB.RAW.EMPLOYEES 
ADD COLUMN 
  FILENAME string, 
  FILE_LAST_MODIFIED timestamp_ntz, 
  FILE_SCAN_TIME timestamp_ltz;

ALTER TABLE HR_DB.RAW.DEPARTMENT
ADD COLUMN 
  FILENAME string, 
  FILE_LAST_MODIFIED timestamp_ntz, 
  FILE_SCAN_TIME timestamp_ltz;

ALTER TABLE HR_DB.RAW.PERFORMANCE_REVIEWS 
ADD COLUMN 
  FILENAME string, 
  FILE_LAST_MODIFIED timestamp_ntz, 
  FILE_SCAN_TIME timestamp_ltz;

In [ ]:
COPY INTO HR_DB.RAW.EMPLOYEES 
  FROM @HR_DB.RAW.landing_stg/employees.csv
  FILE_FORMAT = 'my_csv_format'
  ON_ERROR = CONTINUE
  MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
  INCLUDE_METADATA = (
    FILENAME=METADATA$FILENAME, 
    FILE_LAST_MODIFIED=METADATA$FILE_LAST_MODIFIED, 
    FILE_SCAN_TIME=METADATA$START_SCAN_TIME);

COPY INTO HR_DB.RAW.DEPARTMENT 
  FROM @HR_DB.RAW.landing_stg/departments.csv
  FILE_FORMAT = 'my_csv_format'
  ON_ERROR = CONTINUE
  MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
  INCLUDE_METADATA = (
    FILENAME=METADATA$FILENAME, 
    FILE_LAST_MODIFIED=METADATA$FILE_LAST_MODIFIED, 
    FILE_SCAN_TIME=METADATA$START_SCAN_TIME);

COPY INTO HR_DB.RAW.PERFORMANCE_REVIEWS
  FROM @HR_DB.RAW.landing_stg/Performance_reviews.csv
  FILE_FORMAT = 'my_csv_format'
  ON_ERROR = CONTINUE
  MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
  INCLUDE_METADATA = (
    FILENAME=METADATA$FILENAME, 
    FILE_LAST_MODIFIED=METADATA$FILE_LAST_MODIFIED, 
    FILE_SCAN_TIME=METADATA$START_SCAN_TIME);

In [ ]:
USE SCHEMA ANALYTICS;

CREATE OR REPLACE DYNAMIC TABLE HR_DB.ANALYTICS.DEPARTMENT_STATS
TARGET_LAG = '20 minutes'
  REFRESH_MODE = auto
  WAREHOUSE = ENTERPRISE_CORTEX_WH
  INITIALIZE = on_create
AS
SELECT 
    e.DEPARTMENT,
    COUNT(*) as employee_count,
    AVG(e.SALARY) as avg_salary,
    MIN(e.SALARY) as min_salary,
    MAX(e.SALARY) as max_salary,
    AVG(p.RATING) as avg_performance_rating
FROM HR_DB.RAW.EMPLOYEES e
LEFT JOIN HR_DB.RAW.PERFORMANCE_REVIEWS p
    ON e.EMPLOYEE_ID = p.EMPLOYEE_ID
GROUP BY 1;

In [ ]:
USE DATABASE SALES_DB;

USE SCHEMA RAW;

CREATE OR REPLACE FILE FORMAT SALES_DB.RAW.my_csv_format
  TYPE = csv
  FIELD_OPTIONALLY_ENCLOSED_BY = '"'
  VALIDATE_UTF8 = TRUE
  PARSE_HEADER = TRUE
  ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE;

-- Auto create table with Schema Detection and enable Schema Evolution
CREATE OR REPLACE TABLE SALES_DB.RAW.CUSTOMERS
    USING TEMPLATE (
        SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
          FROM TABLE(
            INFER_SCHEMA(        
            LOCATION=>'@SALES_DB.RAW.landing_stg/Customers.csv'
            , FILE_FORMAT => 'my_csv_format'
            , IGNORE_CASE => TRUE

            
        )
    ))
ENABLE_SCHEMA_EVOLUTION = TRUE
;

-- Auto create table with Schema Detection and enable Schema Evolution
CREATE OR REPLACE TABLE SALES_DB.RAW.PRODUCTS
    USING TEMPLATE (
        SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
        FROM TABLE(
            INFER_SCHEMA(        
            LOCATION=>'@SALES_DB.RAW.landing_stg/products.csv'
            , FILE_FORMAT => 'my_csv_format'
            , IGNORE_CASE => TRUE
        )
    ))
ENABLE_SCHEMA_EVOLUTION = TRUE;

-- Auto create table with Schema Detection and enable Schema Evolution
CREATE OR REPLACE TABLE SALES_DB.RAW.SALES_TRANSACTIONS
    USING TEMPLATE (
        SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
        FROM TABLE(
            INFER_SCHEMA(        
            LOCATION=>'@SALES_DB.RAW.landing_stg/Sales_Transactions.csv'
            , FILE_FORMAT => 'my_csv_format'
            , IGNORE_CASE => TRUE
        )
    ))
ENABLE_SCHEMA_EVOLUTION = TRUE;


ALTER TABLE SALES_DB.RAW.CUSTOMERS
ADD COLUMN 
  FILENAME string, 
  FILE_LAST_MODIFIED timestamp_ntz, 
  FILE_SCAN_TIME timestamp_ltz;

ALTER TABLE SALES_DB.RAW.PRODUCTS
ADD COLUMN 
  FILENAME string, 
  FILE_LAST_MODIFIED timestamp_ntz, 
  FILE_SCAN_TIME timestamp_ltz;

ALTER TABLE SALES_DB.RAW.SALES_TRANSACTIONS
ADD COLUMN 
  FILENAME string, 
  FILE_LAST_MODIFIED timestamp_ntz, 
  FILE_SCAN_TIME timestamp_ltz;

In [ ]:
COPY INTO SALES_DB.RAW.CUSTOMERS
  FROM @SALES_DB.RAW.landing_stg/customers.csv
  FILE_FORMAT = 'my_csv_format'
  ON_ERROR = CONTINUE
  MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
  INCLUDE_METADATA = (
    FILENAME=METADATA$FILENAME, 
    FILE_LAST_MODIFIED=METADATA$FILE_LAST_MODIFIED, 
    FILE_SCAN_TIME=METADATA$START_SCAN_TIME);

COPY INTO SALES_DB.RAW.PRODUCTS 
  FROM @SALES_DB.RAW.landing_stg/products.csv
  FILE_FORMAT = 'my_csv_format'
  ON_ERROR = CONTINUE
  MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
  INCLUDE_METADATA = (
    FILENAME=METADATA$FILENAME, 
    FILE_LAST_MODIFIED=METADATA$FILE_LAST_MODIFIED, 
    FILE_SCAN_TIME=METADATA$START_SCAN_TIME);

COPY INTO SALES_DB.RAW.SALES_TRANSACTIONS
  FROM @SALES_DB.RAW.landing_stg/Sales_Transactions.csv
  FILE_FORMAT = 'my_csv_format'
  ON_ERROR = CONTINUE
  MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
  INCLUDE_METADATA = (
    FILENAME=METADATA$FILENAME, 
    FILE_LAST_MODIFIED=METADATA$FILE_LAST_MODIFIED, 
    FILE_SCAN_TIME=METADATA$START_SCAN_TIME);

In [ ]:
USE SCHEMA ANALYTICS;

CREATE OR REPLACE DYNAMIC TABLE SALES_DB.ANALYTICS.SALES_PERFORMANCE
TARGET_LAG = '20 minutes'
  REFRESH_MODE = auto
  WAREHOUSE = ENTERPRISE_CORTEX_WH
  INITIALIZE = on_create
AS

SELECT 
    DATE_TRUNC('MONTH', s.SALE_DATE) as month,
    s.REGION,
    s.SALES_REP,
    COUNT(*) as total_transactions,
    SUM(s.AMOUNT) as total_sales,
    AVG(s.AMOUNT) as avg_sale_amount,
    COUNT(DISTINCT s.CUSTOMER_ID) as unique_customers
FROM SALES_DB.RAW.SALES_TRANSACTIONS s
GROUP BY 1, 2, 3;

In [ ]:
USE DATABASE MARKETING_DB;

USE SCHEMA RAW;

CREATE OR REPLACE FILE FORMAT MARKETING_DB.RAW.my_csv_format
  TYPE = csv
  FIELD_OPTIONALLY_ENCLOSED_BY = '"'
  VALIDATE_UTF8 = TRUE
  PARSE_HEADER = TRUE
  ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE;

-- Auto create table with Schema Detection and enable Schema Evolution
CREATE OR REPLACE TABLE MARKETING_DB.RAW.CAMPAIGNS
    USING TEMPLATE (
        SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
          FROM TABLE(
            INFER_SCHEMA(        
            LOCATION=>'@MARKETING_DB.RAW.landing_stg/campaigns.csv'
            , FILE_FORMAT => 'my_csv_format'
            , IGNORE_CASE => TRUE

            
        )
    ))
ENABLE_SCHEMA_EVOLUTION = TRUE
;

-- Auto create table with Schema Detection and enable Schema Evolution
CREATE OR REPLACE TABLE MARKETING_DB.RAW.CAMPAIGN_METRICS
    USING TEMPLATE (
        SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
        FROM TABLE(
            INFER_SCHEMA(        
            LOCATION=>'@MARKETING_DB.RAW.landing_stg/campaign_metrics.csv'
            , FILE_FORMAT => 'my_csv_format'
            , IGNORE_CASE => TRUE
        )
    ))
ENABLE_SCHEMA_EVOLUTION = TRUE;


ALTER TABLE MARKETING_DB.RAW.CAMPAIGNS
ADD COLUMN 
  FILENAME string, 
  FILE_LAST_MODIFIED timestamp_ntz, 
  FILE_SCAN_TIME timestamp_ltz;

ALTER TABLE MARKETING_DB.RAW.CAMPAIGN_METRICS
ADD COLUMN 
  FILENAME string, 
  FILE_LAST_MODIFIED timestamp_ntz, 
  FILE_SCAN_TIME timestamp_ltz;


In [ ]:
COPY INTO MARKETING_DB.RAW.CAMPAIGNS
  FROM @MARKETING_DB.RAW.landing_stg/campaigns.csv
  FILE_FORMAT = 'my_csv_format'
  ON_ERROR = CONTINUE
  MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
  INCLUDE_METADATA = (
    FILENAME=METADATA$FILENAME, 
    FILE_LAST_MODIFIED=METADATA$FILE_LAST_MODIFIED, 
    FILE_SCAN_TIME=METADATA$START_SCAN_TIME);

COPY INTO MARKETING_DB.RAW.campaign_metrics 
  FROM @MARKETING_DB.RAW.landing_stg/campaign_metrics.csv
  FILE_FORMAT = 'my_csv_format'
  ON_ERROR = CONTINUE
  MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
  INCLUDE_METADATA = (
    FILENAME=METADATA$FILENAME, 
    FILE_LAST_MODIFIED=METADATA$FILE_LAST_MODIFIED, 
    FILE_SCAN_TIME=METADATA$START_SCAN_TIME);


In [ ]:
USE SCHEMA ANALYTICS;

CREATE OR REPLACE DYNAMIC TABLE MARKETING_DB.ANALYTICS.CAMPAIGN_PERFORMANCE
TARGET_LAG = '20 minutes'
  REFRESH_MODE = auto
  WAREHOUSE = ENTERPRISE_CORTEX_WH
  INITIALIZE = on_create
AS
SELECT 
    c.CAMPAIGN_NAME,
    c.CHANNEL,
    c.STATUS,
    c.BUDGET,
    SUM(m.IMPRESSIONS) as total_impressions,
    SUM(m.CLICKS) as total_clicks,
    SUM(m.CONVERSIONS) as total_conversions,
    SUM(m.SPEND) as total_spend,
    DATEDIFF('DAY', c.START_DATE, c.END_DATE) as campaign_duration
FROM MARKETING_DB.RAW.CAMPAIGNS c
LEFT JOIN MARKETING_DB.RAW.CAMPAIGN_METRICS m
    ON c.CAMPAIGN_ID = m.CAMPAIGN_ID
GROUP BY 1, 2, 3, 4, 9;

In [ ]:
select * from MARKETING_DB.ANALYTICS.CAMPAIGN_PERFORMANCE LIMIT 10;

In [ ]:
ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION = 'AWS_US';

In [ ]:
USE DATABASE ENTERPRISE_DATA_DB;

USE SCHEMA ENTERPRISE_DATA_SCHEMA;

CREATE OR REPLACE TABLE ENTERPRISE_DATA_DB.ENTERPRISE_DATA_SCHEMA.PARSED_DOCUMENTS AS 
SELECT 
    relative_path,
    upper(SPLIT_PART(relative_path, '/', 1)) as department,
    TO_VARCHAR(
        SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
            @landing_stg, 
            relative_path, 
            {'mode': 'LAYOUT'}
        ) :content
    ) AS parsed_text
FROM directory(@landing_stg)
WHERE relative_path ILIKE '%.pdf';

In [ ]:
SELECT * FROM ENTERPRISE_DATA_DB.ENTERPRISE_DATA_SCHEMA.PARSED_DOCUMENTS limit 10;

In [ ]:
    -- Create a table for chunked documents
CREATE OR REPLACE TABLE  ENTERPRISE_DATA_DB.ENTERPRISE_DATA_SCHEMA.CHUNKED_DOCUMENTS (
    FILE_NAME VARCHAR,
    DEPARTMENT VARCHAR,
    CHUNK VARCHAR
);

-- Insert chunked content from parsed documents
INSERT INTO  ENTERPRISE_DATA_DB.ENTERPRISE_DATA_SCHEMA.CHUNKED_DOCUMENTS (FILE_NAME, DEPARTMENT, CHUNK)
SELECT
    relative_path,
    department,
    c.value AS CHUNK
FROM
    PARSED_DOCUMENTS,
    LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
        parsed_text,
        'markdown',
        1800,
        250
    )) c;


In [ ]:
select * from ENTERPRISE_DATA_DB.ENTERPRISE_DATA_SCHEMA.CHUNKED_DOCUMENTS limit 10;

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE ENTERPRISE_DATA_DB.ENTERPRISE_DATA_SCHEMA.DOCUMENT_SEARCH_SERVICE
    ON CHUNK
    ATTRIBUTES (DEPARTMENT)
    WAREHOUSE = ENTERPRISE_CORTEX_WH
    TARGET_LAG = '1 minute'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
    AS (
    SELECT
        *
    FROM ENTERPRISE_DATA_DB.ENTERPRISE_DATA_SCHEMA.CHUNKED_DOCUMENTS
    );


In [ ]:
describe cortex search service ENTERPRISE_DATA_DB.ENTERPRISE_DATA_SCHEMA.DOCUMENT_SEARCH_SERVICE

In [ ]:
SELECT PARSE_JSON(
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
      'ENTERPRISE_DATA_DB.ENTERPRISE_DATA_SCHEMA.DOCUMENT_SEARCH_SERVICE',
      '{
        "query": "Uber",
        "columns":[
            "file_name",
            "chunk"
        ],
        "limit":4
      }'
  )
)['results'] as results;